In [1]:
import pandas as pd
import numpy as np
import os
import cv2


In [2]:
test_folder=r"C:\Users\indra\Desktop\DSG PROJECT\BH25\Testing_Data" # path to test folder
test_video_features = np.load(r'C:\Users\indra\Desktop\DSG PROJECT\final models\test_video_features_noiseless_grayscale_resnet50.npy') # path to the extracted test video feature


In [3]:
video_ids = []
for video_id in sorted(
    [f for f in os.listdir(test_folder) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_ids.append(video_id.split('.')[0])
df = pd.DataFrame({
    'video_id': video_ids,
})

# this creates a dataframe with only the video ids in the 1st column

In [6]:
import joblib
element_model = joblib.load(r"C:\Users\indra\Desktop\DSG PROJECT\final models\element_model.pkl")
motion_model = joblib.load(r"C:\Users\indra\Desktop\DSG PROJECT\final models\motion_model.pkl")
power_model = joblib.load(r"C:\Users\indra\Desktop\DSG PROJECT\final models\power_model.pkl")
speed_model = joblib.load(r"C:\Users\indra\Desktop\DSG PROJECT\final models\speed_model.pkl")
summary_modelx = joblib.load(r"C:\Users\indra\Desktop\DSG PROJECT\final models\summary_x_model.pkl")
summary_modely = joblib.load(r"C:\Users\indra\Desktop\DSG PROJECT\final models\summary_y_model.pkl")

# all the trained and saves models are loaded for test prediction

In [7]:
df

,video_id
0,1
1,2
2,3
3,4
4,5
...,...
2995,2996
2996,2997
2997,2998
2998,2999


# element

In [8]:
# load the labelencoder
element_le = joblib.load(r'C:\Users\indra\Desktop\DSG PROJECT\final models\label_encoder_element.pkl')
predictions_encoded = element_model.predict(test_video_features)

# Decoding the predictions using the loaded LabelEncoder
predictions_decoded = element_le.inverse_transform(predictions_encoded)

df["element"]=predictions_decoded # new column of element names is create in df

print("Predictions (decoded):", predictions_decoded)


Predictions (decoded): ['Erde' 'Erde' 'Erde' ... 'Erde' 'Boden' 'Erde']


In [9]:
df

,video_id,element
0,1,Erde
1,2,Erde
2,3,Erde
3,4,Erde
4,5,Erde
...,...,...
2995,2996,Erde
2996,2997,Erde
2997,2998,Erde
2998,2999,Boden


# motion

In [10]:
# load the labelencoder
motion_le = joblib.load(r'C:\Users\indra\Desktop\DSG PROJECT\final models\label_encoder_motion.pkl')

predictions_encoded = motion_model.predict(test_video_features)

# Decoding the predictions using the loaded LabelEncoder
predictions_decoded = motion_le.inverse_transform(predictions_encoded)

df["motion"]=predictions_decoded # new column of element names is created in the df

print("Predictions (decoded):", predictions_decoded)


Predictions (decoded): ['random' 'random' 'linear' ... 'shm' 'linear' 'random']


In [11]:
df

,video_id,element,motion
0,1,Erde,random
1,2,Erde,random
2,3,Erde,linear
3,4,Erde,shm
4,5,Erde,shm
...,...,...,...
2995,2996,Erde,shm
2996,2997,Erde,random
2997,2998,Erde,shm
2998,2999,Boden,linear


# power

In [12]:
# this is the same function used before for power model to extract the brightest colour of the titan in the video
def extract_brightest_color(video_path):
    cap = cv2.VideoCapture(video_path)
    success, frame = cap.read()  # Reads the frames

    if success:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        brightest_pixel = frame_rgb.reshape(-1, 3).max(axis=0)  # Get rgb values of the brightest pixel
        return [int(value) for value in brightest_pixel]  
    else:
        print(f"Could not read video: {video_path}")
        return None
    cap.release()

def update_csv_with_brightest_color(df, video_folder):
    dff = df
    brightest_colors = []

    for video_id in dff['video_id']:
        video_path = f"{video_folder}/{video_id}.mp4"  # checks the video with the video id in the df
        rgb = extract_brightest_color(video_path)
        brightest_colors.append(rgb)

    dff['shape_rgb_values'] = brightest_colors
    return dff

df= update_csv_with_brightest_color(df, test_folder) # on calling the functions a seperate columns with rgb tupple will be created

df[['R', 'G', 'B']] = pd.DataFrame(df['shape_rgb_values'].tolist(), index=df.index) # the RGB values are seperated into diff columns

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df[['R', 'G', 'B']]
X = scaler.fit_transform(X)
# the values are preprocessed with standardscalar before training with Logistic regression for more accuracy

In [13]:
# Load the saved labelencoder
power_le = joblib.load(r'C:\Users\indra\Desktop\DSG PROJECT\final models\label_encoder_power.pkl')
predictions_encoded = power_model.predict(X)

# Decode the predictions using the loaded LabelEncoder
predictions_decoded = power_le.inverse_transform(predictions_encoded)

df["power"]=predictions_decoded # new columns with power values is created

print("Predictions (decoded):", predictions_decoded)


Predictions (decoded): ['rot' 'lila' 'blau' ... 'gelb' 'gelb' 'lila']


In [14]:
df=df.drop(columns=[['shape_rgb_values','R','G','B']]) # the extra columns are droped
df

,video_id,element,motion,power
0,1,Erde,random,rot
1,2,Erde,random,lila
2,3,Erde,linear,blau
3,4,Erde,shm,rot
4,5,Erde,shm,blau
...,...,...,...,...
2995,2996,Erde,shm,rot
2996,2997,Erde,random,lila
2997,2998,Erde,shm,gelb
2998,2999,Boden,linear,gelb


# speed

In [15]:
# Load the saved labelencoder
speed_le = joblib.load(r'C:\Users\indra\Desktop\DSG PROJECT\final models\label_encoder_speed.pkl')

predictions_encoded = speed_model.predict(test_video_features)

# Decode the predictions using the loaded LabelEncoder
predictions_decoded = speed_le.inverse_transform(predictions_encoded)

df["speed"]=predictions_decoded

print("Predictions (decoded):", predictions_decoded)

Predictions (decoded): [6.9 6.9 9.6 ... 9.6 9.6 9.6]


In [16]:
df

,video_id,element,motion,power,speed
0,1,Erde,random,rot,6.9
1,2,Erde,random,lila,6.9
2,3,Erde,linear,blau,9.6
3,4,Erde,shm,rot,6.9
4,5,Erde,shm,blau,9.6
...,...,...,...,...,...
2995,2996,Erde,shm,rot,9.6
2996,2997,Erde,random,lila,9.6
2997,2998,Erde,shm,gelb,9.6
2998,2999,Boden,linear,gelb,9.6


# summary

In [17]:
# prediction the values of x and y
predictions_x = summary_modelx.predict(test_video_features)
predictions_y = summary_modely.predict(test_video_features)

predictions_x = [round(float(pred), 10) for pred in predictions_x] # this is done to convert the values to float32 and upto 10 decimal place
predictions_y = [round(float(pred), 10) for pred in predictions_y] # this is done to convert the values to float32 and upto 10 decimal place

df["video_summary"]=list(zip(predictions_x, predictions_y)) # new column with zipped values of x and y is created

print("Predictions decoded")


Predictions decoded


In [18]:
df # checking the final df

,video_id,element,motion,power,speed,video_summary
0,1,Erde,random,rot,6.9,"(2.0945073, -5.9732791)"
1,2,Erde,random,lila,6.9,"(3.1802054, -3.6987104)"
2,3,Erde,linear,blau,9.6,"(-1.3200939, 1.4579299)"
3,4,Erde,shm,rot,6.9,"(0.9536747, -1.3488529)"
4,5,Erde,shm,blau,9.6,"(-3.2264, 1.5098046)"
...,...,...,...,...,...,...
2995,2996,Erde,shm,rot,9.6,"(0.4470351, 4.084706)"
2996,2997,Erde,random,lila,9.6,"(0.2765653, 1.8423791)"
2997,2998,Erde,shm,gelb,9.6,"(-0.0636273, 2.0954016)"
2998,2999,Boden,linear,gelb,9.6,"(9.1516978, 0.2208351)"


In [19]:
df.to_csv(r"C:\Users\indra\Desktop\DSG PROJECT\BH25\late_submission.csv", index=False) # saving the final df as csv file

# learnt a lot from this project!!!